In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import zipfile
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', 500)
color = sns.color_palette()

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_zip = zipfile.ZipFile('../input/sberbank-russian-housing-market/train.csv.zip')
data = pd.read_csv(df_zip.open('train.csv'), parse_dates = ['timestamp'])
data.shape


In [ ]:
data.head()

In [ ]:
X = data.drop(['price_doc'], axis = 1)
y = data['price_doc']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size = 0.1, random_state = 42)

В этом наборе данных довольно много переменных.

Начнем с исследования целевой переменной - "price_doc". Сначала давайте сделаем точечную диаграмму, чтобы увидеть, есть ли какие-либо выбросы.

In [ ]:
sns.boxplot(x = y_train)


Хвост справа выглядит  большим, но по скольку метрикой ошибок выбрана RMSLE, то в этом нет ничего страшного. 

In [ ]:
sns.displot(data = X_train, x = y_train, kind = 'kde')

Выполним логарифмирование, с целью выровнять хвост.

In [ ]:
sns.displot(data = X_train, x = np.log(y_train.values), kind = 'kde')

Выглядит гораздо лучше. Выборсы сгладились. Теперь нарисуем график

In [ ]:
data['timestamp']

In [ ]:
data['yearmonth'] = data['timestamp'].apply(lambda x: x.strftime('%Y - %m'))
grouped_data = data.groupby('yearmonth')['price_doc'].aggregate(np.median).reset_index()
plt.figure(figsize=(12,8))
sns.barplot(grouped_data.yearmonth.values, grouped_data.price_doc.values,color='y', alpha=0.8)
plt.ylabel('Медианная цена', fontsize=12)
plt.xlabel('Год Месяц', fontsize=12)
plt.xticks(rotation='vertical')
plt.show()

Посмотрим количество пропущенных (na) значений для каждой характеристики

In [ ]:
missing_data = data.isnull().sum(axis = 0).reset_index()
missing_data.columns = ['column_name', 'missing_count']
missing_df = missing_data.loc[lambda df: df['missing_count'] > 0, :]
ind = np.arange(missing_df.shape[0])
width = 0.9
fig, ax = plt.subplots(figsize=(12,18))
rects = ax.barh(ind, missing_df.missing_count.values, color='y')
ax.set_yticks(ind)
ax.set_yticklabels(missing_df.column_name.values, rotation='horizontal')
ax.set_xlabel("Количество пропусков")
ax.set_title("Количество пропущенных значений в каждой колонке")
plt.show()

Мы видим, что у нас очень много категориальных переменных. Мы можем перевести их в числовые при помощи label encoding

In [ ]:
from sklearn import preprocessing
for f in X_train.columns:
    if X_train[f].dtype=='object':
        print(f)
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(X_train[f].values.astype('str')) + list(X_test[f].values.astype('str')))
        X_train[f] = lbl.transform(list(X_train[f].values.astype('str')))
        X_test[f] = lbl.transform(list(X_test[f].values.astype('str')))

Также у нас очень много пропущенных (na) значений. Заполним их какой-нибуть цифрой лежащей далеко за пределами значений, например -99.

In [ ]:
X_train.fillna(-99, inplace=True)
X_test.fillna(-99, inplace=True)

Посмотрим на отношение цены ко времени, месяцам, дням недели и т.д.

In [ ]:
# year and month https://pandas.pydata.org/pandas-docs/stable/reference/index.html#
X_train["yearmonth"] = X_train["timestamp"].dt.year*100 + X_train["timestamp"].dt.month
X_test["yearmonth"] = X_test["timestamp"].dt.year*100 + X_test["timestamp"].dt.month

# year and week #
X_train["yearweek"] = X_train["timestamp"].dt.year*100 + X_train["timestamp"].dt.weekofyear
X_test["yearweek"] = X_test["timestamp"].dt.year*100 + X_test["timestamp"].dt.weekofyear

# year #
X_train["year"] = X_train["timestamp"].dt.year
X_test["year"] = X_test["timestamp"].dt.year

# month of year #
X_train["month_of_year"] = X_train["timestamp"].dt.month
X_test["month_of_year"] = X_test["timestamp"].dt.month

# week of year #
X_train["week_of_year"] = X_train["timestamp"].dt.weekofyear
X_test["week_of_year"] = X_test["timestamp"].dt.weekofyear

# day of week #
X_train["day_of_week"] = X_train["timestamp"].dt.weekday
X_test["day_of_week"] = X_test["timestamp"].dt.weekday


plt.figure(figsize=(12,8))
sns.pointplot(x='yearweek', y=y_train, data=X_train)
plt.ylabel('Цена', fontsize=12)
plt.xlabel('Месяц, год', fontsize=12)
plt.title('Распределение медиан цен по месяцам года')
plt.xticks(rotation='vertical')
plt.show()

plt.figure(figsize=(12,8))
sns.boxplot(x='month_of_year', y=y_train, data=X_train)
plt.ylabel('Цена', fontsize=12)
plt.xlabel('Месяц', fontsize=12)
plt.title('Распределение медиан цен по месяцам года')
plt.xticks(rotation='vertical')
plt.show()

plt.figure(figsize=(12,8))
sns.pointplot(x='week_of_year', y=y_train, data=X_train)
plt.ylabel('Цена', fontsize=12)
plt.xlabel('Неделя года', fontsize=12)
plt.title('Распределение медиан цен по месяцам года')
plt.xticks(rotation='vertical')
plt.show()

plt.figure(figsize=(12,8))
sns.boxplot(x='day_of_week', y=y_train, data=X_train)
plt.ylabel('Цена', fontsize=12)
plt.xlabel('День недели', fontsize=12)
plt.title('Распределение медианных цен по дням недели')
plt.xticks(rotation='vertical')
plt.show()

Как ни странно, но какая-то линейная зависимость по месяцу есть. Недвижимость немного растет в цене к концу года

Посмотрим, какие характеристики являются наиболее важными.

In [ ]:
import xgboost as xgb
if X_train[f].dtype=='object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(train_df[f].values)) 
        X_train[f] = lbl.transform(list(X_train[f].values))
        
X_train = X_train.drop(["id", "timestamp"], axis=1)

xgb_params = {
    'eta': 0.05,
    'max_depth': 8,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 1
}
dtrain = xgb.DMatrix(X_train, y_train, feature_names=X_train.columns.values)
model = xgb.train(dict(xgb_params, silent=0), dtrain, num_boost_round=100)

# plot the important features #
fig, ax = plt.subplots(figsize=(12,18))
xgb.plot_importance(model, max_num_features=50, height=0.8, ax=ax)
plt.show()

На самом деле, можно было бы выполнить анализ уже на данном этапе, однако с целью улучшения точности, мы создадим несколько новых переменных.

Наглядно видно, что конечная стоимость сильнее всего зависит от площади. Попробуем создать несколько переменных соотношения к площади, которые могли бы внести больше ясности.

In [ ]:
# Отношения жилой площади ко всей #
X_train["ratio_life_sq_full_sq"] = X_train["life_sq"] / np.maximum(X_train["full_sq"].astype("float"),1)
X_test["ratio_life_sq_full_sq"] = X_test["life_sq"] / np.maximum(X_test["full_sq"].astype("float"),1)
X_train["ratio_life_sq_full_sq"].loc[X_train["ratio_life_sq_full_sq"]<0] = 0
X_train["ratio_life_sq_full_sq"].loc[X_train["ratio_life_sq_full_sq"]>1] = 1
X_test["ratio_life_sq_full_sq"].loc[X_test["ratio_life_sq_full_sq"]<0] = 0
X_test["ratio_life_sq_full_sq"].loc[X_test["ratio_life_sq_full_sq"]>1] = 1

# Отношение площади кухни к жилой #
X_train["ratio_kitch_sq_life_sq"] = X_train["kitch_sq"] / np.maximum(X_train["life_sq"].astype("float"),1)
X_test["ratio_kitch_sq_life_sq"] = X_test["kitch_sq"] / np.maximum(X_test["life_sq"].astype("float"),1)
X_train["ratio_kitch_sq_life_sq"].loc[X_train["ratio_kitch_sq_life_sq"]<0] = 0
X_train["ratio_kitch_sq_life_sq"].loc[X_train["ratio_kitch_sq_life_sq"]>1] = 1
X_test["ratio_kitch_sq_life_sq"].loc[X_test["ratio_kitch_sq_life_sq"]<0] = 0
X_test["ratio_kitch_sq_life_sq"].loc[X_test["ratio_kitch_sq_life_sq"]>1] = 1

# Отношение площади кухни ко всей #
X_train["ratio_kitch_sq_full_sq"] = X_train["kitch_sq"] / np.maximum(X_train["full_sq"].astype("float"),1)
X_test["ratio_kitch_sq_full_sq"] = X_test["kitch_sq"] / np.maximum(X_test["full_sq"].astype("float"),1)
X_train["ratio_kitch_sq_full_sq"].loc[X_train["ratio_kitch_sq_full_sq"]<0] = 0
X_train["ratio_kitch_sq_full_sq"].loc[X_train["ratio_kitch_sq_full_sq"]>1] = 1
X_test["ratio_kitch_sq_full_sq"].loc[X_test["ratio_kitch_sq_full_sq"]<0] = 0
X_test["ratio_kitch_sq_full_sq"].loc[X_test["ratio_kitch_sq_full_sq"]>1] = 1

plt.figure(figsize=(12,12))
sns.jointplot(x=X_train.ratio_life_sq_full_sq.values, y=np.log1p(y_train.values), size=10)
plt.ylabel('Log of Price', fontsize=12)
plt.xlabel('Ratio of living area to full area', fontsize=12)
plt.title("Joint plot on log of living price to ratio_life_sq_full_sq")
plt.show()

plt.figure(figsize=(12,12))
sns.jointplot(x=X_train.ratio_life_sq_full_sq.values, y=np.log1p(y_train.values), 
              kind='kde',size=10)
plt.ylabel('Log of Price', fontsize=12)
plt.xlabel('Ratio of kitchen area to living area', fontsize=12)
plt.title("Joint plot on log of living price to ratio_kitch_sq_life_sq")
plt.show()

plt.figure(figsize=(12,12))
sns.jointplot(x=X_train.ratio_life_sq_full_sq.values, y=np.log1p(y_train.values), 
              kind='kde',size=10)
plt.ylabel('Log of Price', fontsize=12)
plt.xlabel('Ratio of kitchen area to full area', fontsize=12)
plt.title("Joint plot on log of living price to ratio_kitch_sq_full_sq")
plt.show()

Также важными переменными являются floor и max_floor. Давайте создадим две переменных

Отношение этажа к макс. количеству этажей
Количество этажей сверху

In [ ]:
# floor of the house to the total number of floors in the house #
X_train["ratio_floor_max_floor"] = X_train["floor"] / X_train["max_floor"].astype("float")
X_test["ratio_floor_max_floor"] = X_test["floor"] / X_test["max_floor"].astype("float")

# num of floor from top #
X_train["floor_from_top"] = X_train["max_floor"] - X_train["floor"]
X_test["floor_from_top"] = X_test["max_floor"] - X_test["floor"]

Еще одной переменной, зависящей от площади этажа, может быть разность между полной площадью и жилой площадью.

In [ ]:
X_train["extra_sq"] = X_train["full_sq"] - X_train["life_sq"]
X_test["extra_sq"] = X_test["full_sq"] - X_test["life_sq"]

Возраст здания может повлиять на стоимость аренды, поэтому мы должны создать и эту переменную.

In [ ]:
X_train["age_of_building"] = X_train["build_year"] - X_train["year"]
X_test["age_of_building"] = X_test["build_year"] - X_test["year"]

Школы обычно играют большое значение в выборе дома, так-что создадим несколько переменных и вокруг школ.

In [ ]:
X_train["ratio_preschool"] = X_train["children_preschool"] / X_train["preschool_quota"].astype("float")
X_test["ratio_preschool"] = X_test["children_preschool"] / X_test["preschool_quota"].astype("float")

X_train["ratio_school"] = X_train["children_school"] / X_train["school_quota"].astype("float")
X_test["ratio_school"] = X_test["children_school"] / X_test["school_quota"].astype("float")

Потенциально мы могли бы добавить больше переменных, подобных этой. Но мы уже обошли наиболее значимые, дальнейшее добавление переменных создаст лишний шум. Давайте сейчас начнем с построения модели с использованием этих уже созданных переменных. 

In [ ]:
X_train.replace([np.inf, -np.inf], np.nan, inplace=True)
X_test.replace([np.inf, -np.inf], np.nan, inplace=True)

In [ ]:
X_train.fillna(-99, inplace=True)
X_test.fillna(-99, inplace=True)

In [ ]:
for f in X_train.columns:
    if X_train[f].dtype != 'float' and X_train[f].dtype != 'int':
        print(f)

Поскольку наша метрика "RMSLE", давайте использовать логарифмированную целевую переменную для построения модели, а не фактическую.

In [ ]:
#y_train = np.log1p(y_train)
#y_test = np.log1p(y_test)

In [ ]:
X_test = X_test.drop(["id", "timestamp"], axis=1)

xgb_params = {
    'eta': 0.05,
    'max_depth': 4,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'min_child_weight':1,
    'silent': 1,
    'seed':0
}

xgtrain = xgb.DMatrix(X_train, y_train, feature_names=X_train.columns)
xgtest = xgb.DMatrix(X_test, y_test, feature_names=X_test.columns)
watchlist = [ (xgtrain,'train'), (xgtest, 'test') ]
num_rounds = 100 # Increase the number of rounds while running in local
model = xgb.train(xgb_params, xgtrain, num_rounds, watchlist, early_stopping_rounds=50, verbose_eval=5)

In [ ]:
fig, ax = plt.subplots(figsize=(12,18))
xgb.plot_importance(model, max_num_features=50, height=0.8, ax=ax)
plt.show()

In [ ]:
y_predict = model.predict(xgtrain)
output = pd.DataFrame({ 'price_doc': y_predict})
output.head()

In [ ]:
print(y_test.head())